## load orig data

In [1]:
import pandas as pd
import numpy as np

train_path = "./dataset/dataset-train.inter" 
valid_path = "./dataset/dataset-valid.inter"
test_path = "./dataset/dataset-test.inter"
udoc_path = "./dataset/zhilian.udoc"
idoc_path = "./dataset/zhilian.idoc"

train_interact_df = pd.read_csv(train_path, sep="\t")
valid_interact_df = pd.read_csv(valid_path, sep="\t")
test_interact_df = pd.read_csv(test_path, sep="\t")

udoc_df = pd.read_csv(udoc_path, sep="\t")
idoc_df = pd.read_csv(idoc_path, sep="\t", on_bad_lines='warn')

KeyboardInterrupt: 

In [138]:
train_df = train_interact_df.merge(idoc_df, on="job_id:token", how="inner").merge(udoc_df, on="user_id:token", how="inner")
valid_df = valid_interact_df.merge(idoc_df, on="job_id:token", how="inner").merge(udoc_df, on="user_id:token", how="inner")
test_df = test_interact_df.merge(idoc_df, on="job_id:token", how="inner").merge(udoc_df, on="user_id:token", how="inner")
# str(train_df.iloc[[0]].to_dict(orient="index")[0]['cur_jd_type:token_seq'])

display(train_df.describe())
display(valid_df.describe())
display(test_df.describe())

,browsed:label,delivered:label,satisfied:label
count,511515.000000,511515.000000,511515.000000
mean,0.216297,0.094996,0.044278
std,0.411719,0.293210,0.205713
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


,browsed:label,delivered:label,satisfied:label
count,63950.000000,63950.000000,63950.000000
mean,0.214167,0.093917,0.043096
std,0.410247,0.291716,0.203075
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


,browsed:label,delivered:label,satisfied:label
count,63941.000000,63941.000000,63941.000000
mean,0.212821,0.093258,0.044056
std,0.409305,0.290796,0.205222
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


## Duplicate Removal

In [139]:
train_df = train_df.drop_duplicates(subset=["user_id:token", "job_id:token"])
valid_df = valid_df.drop_duplicates(subset=["user_id:token", "job_id:token"])
test_df = test_df.drop_duplicates(subset=["user_id:token", "job_id:token"])

## Statistics

In [143]:
train_df

,user_id:token,job_id:token,browsed:label,delivered:label,satisfied:label,job_doc:token_seq,desire_jd_industry_id:token_seq,desire_jd_type_id:token_seq,cur_industry_id:token_seq,cur_jd_type:token_seq,experience:token_seq
0,125f691de44821d5d170b0ef3f59de57,ed2a4741f3254dfc6020b0f0be715927,0,0,0,1、法学专业本科；2、能够熟练操作办公软件；草拟法律文书，资料整理、数据汇总3、工作踏实勤奋...,"专业服务/咨询(财会/法律/人力资源等),保险,教育/培训/院校",兼职,保险,NaN,法务|调查|自我评价|财险|合规|律师
1,25c0d95add228a30a6453448b6e2b296,f99825d37c3c7d853b0f4f31264d0b25,0,0,0,1、所负责区域内教育主管单位及学校的关系维护；2、基于产品及用户开展培训及运营工作（拓校、活...,教育/培训/院校,"行政/后勤/文秘,人力资源",贸易/进出口,NaN,讲解|英语老师|行政|辅导|生产|检查|办公室主任|教育培训|学生|文书|安全|煤矿|办公室...
2,05f5d36ed65cc7204ac8a6edfa6df36f,3ee1f23ca787d127fc86ee5bd8a3ace3,0,0,0,岗位职责:1、负责经销商及经销商业务人员的管理与培训； 2、负责公司各项销售活动在终端的推...,"房地产/建筑/建材/工程,物业管理/商业中心,物流/仓储","房地产开发/经纪/中介,物业管理,土木/建筑/装修/市政工程",房地产/建筑/建材/工程,房地产开发/经纪/中介,计划|预算|综合|银行|按揭|处理|office|接待|房地产|合同|手续|成本|建造师|传...
3,0849ae1528bd1f22c0e32d6a3469cec7,3e2e7dd3c55a9e5799b8865c04896135,0,0,0,1.20-25岁，160cm以上，性格开朗随和，有亲和力，衣着整齐大方得体。2.普通话标准。...,教育/培训/院校,"小学教师,外语教师,培训助理/助教",教育/培训/院校,NaN,活动|教师资格|普通话|外国语言文学|教师|辅导|老师|前台|普通话等级证|接待|教育培训|...
4,07068262c856c973916a6401d0a7669e,addde24d60b96765233eece119d4e8d0,0,0,0,岗位职责：1.负责部门文档（资料、档案等）的收集、整理、归档、借阅、盘点等工作；2.负责将档...,"贸易|批发|零售|租赁业,贸易/进出口,物流/仓储","销售业务,仓库/物料管理员",房地产/建筑/建材/工程,医院/医疗/护理,档案管理|事务管理|接待|组织|保证|公文|促销|协调|药房|面试|药品|安排|库存|协助|...
...,...,...,...,...,...,...,...,...,...,...,...
511507,0c10d9557adc9cf61910d97bfbbdf209,bb2e773bbee834fcaf9d6e4a94887ca4,0,0,0,岗位职责：1、完成公司各类视频、音频剪辑需求，包含但不限于公司内部宣传片、教师上课视频等内容...,交通/运输,"银行会计/柜员,公交/地铁乘务,列车驾驶/操作",教育/培训/院校,NaN,after effects（ae）|视频音频制作|视频剪辑|premiere（pr）剪辑|证...
511508,0f9f6a76c6674cecb136df7540a85c9e,175aa25a098bd5da9e2f69c1c789ed3a,0,0,0,岗位职责：1、负责维护人事信息系统内人事数据的及时、准确、完整性，定期分析人事数据，提供相关...,媒体/出版/影视/文化传播,平面设计,教育/培训/院校,NaN,互联网参考模型osi七层|报税|人力资源专员|平面设计|网页制作|广告设计|h5|人力资源3...
511511,0ac95ab0a4278eddda27abdb0c5ea5ca,9febf46d62766a53609dae78d8dc02b6,0,0,0,深圳市盐田区政府行政服务大厅招聘:22名窗口服务人员，从事盐田辖区行政服务大厅窗口工作，具体...,"广告/会展/公关,媒体/出版/影视/文化传播,教育/培训/院校","广告/会展,公关/媒介,影视/媒体/出版/印刷",文化|传媒|娱乐|体育,公关/媒介,赛事|图片|现场|传播|脚本撰写|机构|导师|完美|处理|媒体|组织|保证|条理性|策划执行...
511512,0247bbd12fb01609d049abe3d818f604,d8591738135318317be0b2a8f8efee53,1,0,0,岗位职责简述：1、负责编制项目《工程策划报告》，布置现场总平面，工程施工总体进度计划；2、负...,房地产/建筑/建材/工程,"城市规划与设计,房地产项目管理,房地产项目策划经理/主管",房地产/建筑/建材/工程,房地产开发/经纪/中介,体系|模式|建设|开发流程|安置|设计规划|规划设计|经理|乡村|基金|旅游|苗木|智慧|国...


In [140]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(392912, 11)
(61381, 11)
(61440, 11)


In [141]:
# browsed:label, delivered:label, satisfied:label pos(1), neg(0) sample ratio
from typing import List
def pos_neg_sample_ratio(df, label_fields: List[str]):
    """
    df: one of train_df, valid_df, test_df;
    label_field: one or more of ["browsed:label", "delivered:label", "satisfied:label"]
    return:
    a dataframe with pos/neg sample ratio for each label_field
    """
    ans = dict()
    for label_field in label_fields:
        pos = df[label_field].sum()
        neg = len(df) - pos
        ans[label_field] = {
            "pos/neg": pos / neg,
            "pos/total": pos / df.shape[0],
            "neg/total": neg / df.shape[0]
        }
    ans = df.from_dict(ans, orient="index")
    return ans

print("train_df pos/neg sample ratio:\n", pos_neg_sample_ratio(train_df, ["browsed:label", "delivered:label", "satisfied:label"]))
print("valid_df pos/neg sample ratio:\n", pos_neg_sample_ratio(valid_df, ["browsed:label", "delivered:label", "satisfied:label"]))
print("test_df pos/neg sample ratio:\n", pos_neg_sample_ratio(test_df, ["browsed:label", "delivered:label", "satisfied:label"]))

train_df pos/neg sample ratio:
                   pos/neg  pos/total  neg/total
browsed:label    0.297789   0.229459   0.770541
delivered:label  0.124604   0.110798   0.889202
satisfied:label  0.053039   0.050368   0.949632
valid_df pos/neg sample ratio:
                   pos/neg  pos/total  neg/total
browsed:label    0.275767   0.216158   0.783842
delivered:label  0.106722   0.096430   0.903570
satisfied:label  0.046261   0.044216   0.955784
test_df pos/neg sample ratio:
                   pos/neg  pos/total  neg/total
browsed:label    0.274134   0.215153   0.784847
delivered:label  0.105832   0.095703   0.904297
satisfied:label  0.047195   0.045068   0.954932


In [142]:
# for a user, how many jobs he/she has browsed/delivered/satisfied.
user_funnel_train = train_df.groupby("user_id:token")[['browsed:label', 'delivered:label', 'satisfied:label']].sum()
user_funnel_valid = valid_df.groupby("user_id:token")[['browsed:label', 'delivered:label', 'satisfied:label']].sum()
user_funnel_test = test_df.groupby("user_id:token")[['browsed:label', 'delivered:label', 'satisfied:label']].sum()

# display(user_funnel_train.head(10))
# display(user_funnel_valid.head(10))
# display(user_funnel_test.head(10))

# for a job, how many users is satisfied by the hr?
job_funnel_train = train_df.groupby("job_id:token")[['satisfied:label']].sum()
job_funnel_valid = valid_df.groupby("job_id:token")[['satisfied:label']].sum()
job_funnel_test = test_df.groupby("job_id:token")[['satisfied:label']].sum()


job_funnel_train['satisfied:label'].unique()
# display(job_funnel_train)
# display(job_funnel_valid)
# display(job_funnel_test)
# Calculate On Average, how many candidates job only receives
print("job_funnel_train.mean(): ",job_funnel_train.mean())
print("job_funnel_valid.mean(): ",job_funnel_valid.mean())
print("job_funnel_test.mean(): ",job_funnel_test.mean())

job_funnel_train.mean():  satisfied:label    0.0912
dtype: float64
job_funnel_valid.mean():  satisfied:label    0.05623
dtype: float64
job_funnel_test.mean():  satisfied:label    0.057533
dtype: float64


## Preprocess

In [118]:
def gen_udoc_idoc_label(df, job_field='job_doc:token_seq'):
    """
    Generate cv and jd text representations from dataframe
    
    Args:
        df: Input dataframe
        job_field: Column name containing job description
    
    Returns:
        DataFrame with cv, jd and label columns
    """
    new_df = df.copy()

    
    
    def gen_cv(row):
        cv_parts = ["个人简历: "]
        
        # Field mappings with their headers
        field_mappings = {
            "experience:token_seq": "【工作经历】 ",
            "desire_jd_industry_id:token_seq": "【期望行业】: ",
            "cur_industry_id:token_seq": "【当前行业】",
            "jd_type_id:token_seq": "【职位标签:】"
        }
        
        # Add each field with its header
        for field, header in field_mappings.items():
            if field in row:
                cv_parts.append(header)
                cv_parts.append(str(row[field]) if pd.notna(row[field]) else "无 ")
        
        return " ".join(cv_parts)
    
    def gen_jd(row):
        return str(row[job_field]) if pd.notna(row[job_field]) else ""
    
    # Generate cv and jd columns
    new_df['cv'] = new_df.apply(gen_cv, axis=1)
    new_df['jd'] = new_df.apply(gen_jd, axis=1)
    
    # Return relevant columns
    label_fields = ["browsed:label", "delivered:label", "satisfied:label"]
    return new_df[["cv", "jd"] + label_fields]
processed_df_train = gen_udoc_idoc_label(train_df)
processed_df_valid = gen_udoc_idoc_label(valid_df)
processed_df_test = gen_udoc_idoc_label(test_df)

In [119]:
display(idoc_df.isnull().mean())
display(udoc_df.isnull().mean())

job_id:token         0.0
job_doc:token_seq    0.0
dtype: float64

user_id:token                      0.000000
desire_jd_industry_id:token_seq    0.000000
desire_jd_type_id:token_seq        0.000000
cur_industry_id:token_seq          0.004253
cur_jd_type:token_seq              0.376539
experience:token_seq               0.000000
dtype: float64

In [121]:
train_interact_df.merge(idoc_df, on="job_id:token", how="left").isnull().mean()

user_id:token        0.00000
job_id:token         0.00000
browsed:label        0.00000
delivered:label      0.00000
satisfied:label      0.00000
job_doc:token_seq    0.00907
dtype: float64

In [122]:
train_interact_df.merge(udoc_df, on="user_id:token", how="left").isnull().mean()

user_id:token                      0.000000
job_id:token                       0.000000
browsed:label                      0.000000
delivered:label                    0.000000
satisfied:label                    0.000000
desire_jd_industry_id:token_seq    0.000000
desire_jd_type_id:token_seq        0.000000
cur_industry_id:token_seq          0.003464
cur_jd_type:token_seq              0.370341
experience:token_seq               0.000000
dtype: float64

In [123]:
pd.merge(train_interact_df, idoc_df, on="job_id:token", how="left")

,user_id:token,job_id:token,browsed:label,delivered:label,satisfied:label,job_doc:token_seq
0,125f691de44821d5d170b0ef3f59de57,ed2a4741f3254dfc6020b0f0be715927,0,0,0,1、法学专业本科；2、能够熟练操作办公软件；草拟法律文书，资料整理、数据汇总3、工作踏实勤奋...
1,25c0d95add228a30a6453448b6e2b296,f99825d37c3c7d853b0f4f31264d0b25,0,0,0,1、所负责区域内教育主管单位及学校的关系维护；2、基于产品及用户开展培训及运营工作（拓校、活...
2,05f5d36ed65cc7204ac8a6edfa6df36f,3ee1f23ca787d127fc86ee5bd8a3ace3,0,0,0,岗位职责:1、负责经销商及经销商业务人员的管理与培训； 2、负责公司各项销售活动在终端的推...
3,0849ae1528bd1f22c0e32d6a3469cec7,3e2e7dd3c55a9e5799b8865c04896135,0,0,0,1.20-25岁，160cm以上，性格开朗随和，有亲和力，衣着整齐大方得体。2.普通话标准。...
4,07068262c856c973916a6401d0a7669e,addde24d60b96765233eece119d4e8d0,0,0,0,岗位职责：1.负责部门文档（资料、档案等）的收集、整理、归档、借阅、盘点等工作；2.负责将档...
...,...,...,...,...,...,...
516192,005ee1b3219a6e71301a7d8ec9399d3f,d0429566419c44b72141b8b4daa9ec05,0,0,0,岗位职责:1、根据市场情况合理分配目标，完成公司下达的销售目标2、根据公司的市场发展策略，制...
516193,0ac95ab0a4278eddda27abdb0c5ea5ca,9febf46d62766a53609dae78d8dc02b6,0,0,0,深圳市盐田区政府行政服务大厅招聘:22名窗口服务人员，从事盐田辖区行政服务大厅窗口工作，具体...
516194,0247bbd12fb01609d049abe3d818f604,d8591738135318317be0b2a8f8efee53,1,0,0,岗位职责简述：1、负责编制项目《工程策划报告》，布置现场总平面，工程施工总体进度计划；2、负...
516195,16505bedd29a4acfdbe53736ffdd08ff,1958b10e859b753b71d111dff568730a,0,0,0,岗位职责： 1.经销商管理：业务沟通和谈判，销售活动组织，供应链管理2.销售目标达成：达成公...


In [124]:
train_df.isnull().mean()

user_id:token                      0.000000
job_id:token                       0.000000
browsed:label                      0.000000
delivered:label                    0.000000
satisfied:label                    0.000000
job_doc:token_seq                  0.000000
desire_jd_industry_id:token_seq    0.000000
desire_jd_type_id:token_seq        0.000000
cur_industry_id:token_seq          0.003670
cur_jd_type:token_seq              0.369653
experience:token_seq               0.000000
dtype: float64

In [125]:
display(train_df[["browsed:label", "delivered:label", "satisfied:label"]].mean())
train_df.iloc[[0]]

browsed:label      0.229459
delivered:label    0.110798
satisfied:label    0.050368
dtype: float64

,user_id:token,job_id:token,browsed:label,delivered:label,satisfied:label,job_doc:token_seq,desire_jd_industry_id:token_seq,desire_jd_type_id:token_seq,cur_industry_id:token_seq,cur_jd_type:token_seq,experience:token_seq
0,125f691de44821d5d170b0ef3f59de57,ed2a4741f3254dfc6020b0f0be715927,0,0,0,1、法学专业本科；2、能够熟练操作办公软件；草拟法律文书，资料整理、数据汇总3、工作踏实勤奋...,"专业服务/咨询(财会/法律/人力资源等),保险,教育/培训/院校",兼职,保险,NaN,法务|调查|自我评价|财险|合规|律师


In [127]:
# load processed
processed_train = pd.read_csv("./dataset/processed_train.csv")
processed_valid = pd.read_csv("./dataset/processed_valid.csv")
processed_test = pd.read_csv("./dataset/processed_test.csv")

processed_train

,user_id:token,job_id:token,cv,jd,browsed:label,delivered:label,satisfied:label
0,125f691de44821d5d170b0ef3f59de57,ed2a4741f3254dfc6020b0f0be715927,个人简历: 【工作经历】 法务|调查|自我评价|财险|合规|律师 【期望行业】: 专业...,1、法学专业本科；2、能够熟练操作办公软件；草拟法律文书，资料整理、数据汇总3、工作踏实勤奋...,0,0,0
1,125f691de44821d5d170b0ef3f59de57,a360fa7511bbfa7d270fa60f0a71d4ba,个人简历: 【工作经历】 法务|调查|自我评价|财险|合规|律师 【期望行业】: 专业...,岗位职责：1、协助指导律师做好工作计划、工作安排；2、协助律师开展工作，如为诉讼案件，协助律...,0,0,0
2,125f691de44821d5d170b0ef3f59de57,fd216a28fa949380f14965b63076e544,个人简历: 【工作经历】 法务|调查|自我评价|财险|合规|律师 【期望行业】: 专业...,岗位职责：1、通过系统及时把订单推送给仓库；2、审核、校对订单，处理店铺未发货异常订单，确保...,0,0,0
3,125f691de44821d5d170b0ef3f59de57,af11b664039e6f347681b14a0837702a,个人简历: 【工作经历】 法务|调查|自我评价|财险|合规|律师 【期望行业】: 专业...,岗位职责1、准备：（1）法律法规研究准备：接受到任何案件或客户的法律问题，首先协助指导律师做...,0,0,0
4,125f691de44821d5d170b0ef3f59de57,58e2b913c253385e775211047017a590,个人简历: 【工作经历】 法务|调查|自我评价|财险|合规|律师 【期望行业】: 专业...,岗位职责：1.对公司合作方提供的视频素材进行核查，确定视频尺度、可观赏度、受众程度、敏感度等...,0,0,0
...,...,...,...,...,...,...,...
392907,0a13162378df7f0f05f49502cf920d3b,3c3520bc65a03d7e127de61e58e34953,个人简历: 【工作经历】 银行|资料|电话|做账|总账会计|财务报表|处理|接待|会计|...,1、负责公司的往来单据核对及录入2、负责开具发票工作；3、跟进客户订单完成情况4.完成领导安...,1,1,1
392908,2749c0be969cb6a24c2a6c2efc5f9f67,f365a33c1493c5166b531878c70bcb5c,个人简历: 【工作经历】 投资|现场|施工|检测|工程质量|测量|吃苦|工程测量|放线|...,1、5年以上工地现场工作经验；2、熟悉土建施工规范、会CAD制图、会根据图纸提量、懂得测量放...,1,0,0
392909,2749c0be969cb6a24c2a6c2efc5f9f67,71017b61725aec86487aa8e80c299648,个人简历: 【工作经历】 投资|现场|施工|检测|工程质量|测量|吃苦|工程测量|放线|...,岗位职责：1.遵循项目部的各项规定和程序，明确施工工艺，制定出每项工作的作业顺序，使施工有条...,1,1,1
392910,058c8581e2e2a4adee62b513270a4ace,128da0e8e8141091e2b7a469d9c21209,个人简历: 【工作经历】 市场营销|园林|中餐烹饪|领导|退伍军人|材料|员工|网络营销...,岗位职责:1、了解客情及菜单内容，明确当日工作任务。2、根据预订原料清单，按质按量及时领取各...,1,1,1


In [128]:
display(processed_train.isnull().mean())
display(processed_valid.isnull().mean())
display(processed_test.isnull().mean())

user_id:token      0.0
job_id:token       0.0
cv                 0.0
jd                 0.0
browsed:label      0.0
delivered:label    0.0
satisfied:label    0.0
dtype: float64

user_id:token      0.0
job_id:token       0.0
cv                 0.0
jd                 0.0
browsed:label      0.0
delivered:label    0.0
satisfied:label    0.0
dtype: float64

user_id:token      0.0
job_id:token       0.0
cv                 0.0
jd                 0.0
browsed:label      0.0
delivered:label    0.0
satisfied:label    0.0
dtype: float64

In [129]:
data = processed_train["user_id:token"].unique()
random_sample = np.random.choice(data, size=10, replace=False)

In [130]:
random_sample

array(['241fd249f30d1514c85e6890c0a7ffef',
       '1209d6e8c1adc0dd56ef75ffd725d74e',
       '29b4c2cf53716c20abbee7b79e05db75',
       '26b0a267a8a6b075bc18493449e6164e',
       '16d95f1606a4b1df66b89f98f52d8ba6',
       '2a27143dc21c1592f3ff9070eebfc82d',
       '008007bfde96cd70f44213e0a7c1af86',
       '2c8c978ac31de106cad7303f61dee5ed',
       '04664e9ed2646b39e8dcf1adb9c85e3e',
       '160ae081aaa4250ee42d812dd88d6bd3'], dtype=object)

In [131]:
sample_records = processed_train[processed_train["user_id:token"].isin(random_sample)]

In [132]:
user_record = sample_records[sample_records["user_id:token"] == random_sample[0]]
user_browsed = user_record[user_record["browsed:label"] == 1]
user_unbrosed = user_record[user_record["browsed:label"] == 0]
print(user_record["cv"].tolist()[0])

个人简历:  【工作经历】  计划|互联网参考模型osi七层|现场|检测|项目施工|网络通讯协议|office|工程施工|组织|设计方案|设备|施工员|建筑|协调|安排|方案|技术|人力|工程测量|主持|土建|团队领导|质量|现场施工|学习能力|进度|配套|检测员|园林|施工|材料|cad|工程|沟通能力|实施|装修|后期|规划|工地|房屋|校园|施工管理|精装修|工程设计|生产|业主|测量|保护|编制|项目经理|地质|安全|市政|前期|适应能力|平面图|工程进度|施工现场 【期望行业】:  房地产/建筑/建材/工程 【当前行业】 房地产/建筑/建材/工程


In [133]:
user_unbrosed["job_id:token"].nunique()

191

In [134]:
user_browsed["jd"].tolist()

['Normal07.8磅02falsefalsefalseEN-USZH-CNX-NONE岗位职责：1、负责对开发项目土建专业的工程进度、质量、安全文明施工情况进行检查和监督，跟踪落实土建专业的工程质量状况；2、参与编制施工控制计划及分项施工进度计划，并督导实施、纠偏；3、负责控制无效成本，及时核实现场签证的工作量;4、负责组织图纸会审，提前发现问题，进行设计变更的执行；5、负责土建专业竣工档案管理，参与项目土建专业重大验收和资料移交工作；6、协调项目总包、分包与监理的关系；协调在项目进程中本专业同其它专业工程师的关系。任职要求：1、本科及以上学历；2.工作经验：8年以上大型房地产企业土建施工及地产开发经验；有甲方完整参项目施工的监督管理工作经验3、能力要求：熟悉项目管理，熟悉施工技术，较强的沟通能力和表达能力；4、项目经验：有大型公建项目的全过程管理经验优先。',
 '岗位职责：1.负责施工过程中土建专业的施工质量、进度、安全的监管；2.以土建专业（建筑、结构、室内外园林景观以及相关专业内容）设计文件为指导，协调现场施工，协调设计单位与施工单位、监理单位的设计资料交底；3.具备发现和解决现场施工和施工管理问题的能力，能够按领导要求及时处理问题；4.监管施工的全过程，负责施工材料的审批和查验并做好协调管理工作，负责监督隐蔽工程的验收和工程的验收；5.公司交办的其他工作。任职资格：1.工民建或土木工程专业本科及以上学历；2.8年以上工作经验（在一家公司持续工作4年以上），并具备不少于4年以上现场技术及施工管理经验；3.有建筑或结构专业设计工作经历者优先考虑，有专业间设计协调能力者优先；4.现场施工管理经验不少于1个建筑面积在5万平米以上工程，且参与了设计至竣工的全过程；5.熟悉混凝土结构及钢结构施工图纸、设计及验收规范，熟悉施工现场工作流程和环节；6.工作严谨、细致、有责任心、能够吃苦耐劳。任何情况下能够以工作为重，以大局为重；7.具有基本的英语水平和计算机操作能力，熟练使用CAD、Office等办公软件。',
 '岗位职责：参与施工组织管理策划、图纸会审、技术核定对施工程序、方案、材料、质量进行跟踪控制落实土建进度计划并协调其他相关专业工程的进度参与招投标工作，为其提供相关技术文件，审核控制预算外签证参与验收，做好相关过程管理资料的收集和整理任职资格：5年以上相关

In [135]:
user_unbrosed["jd"].tolist()

['1、负责土建专业范围的专业施工管理，控制工程质量，组织分项工程的验收；\xa02、编写专业施工进度计划，控制工程进度；\xa03、协调专业工程与相关专业配套、电气工程之间的关系；\xa04、协调承包商与监理公司的关系；\xa05、负责土建类甲供材料的定货、验收等工作；\xa06、制定土建专业部分甲供材料、甲限价材料、甲分包工程计划，配合工程管理部进行有关甲供材料、限价材料的招标和限价工作。\xa01、本科及以上，工民建、结构、土木工程类相关专业；\xa02、5年以上规范房产企业工程管理经验；\xa03、熟悉国家、地方规范及标准，掌握项目规划、设计、施工及验收等基本建设程序；很强的现场管理能力；\xa04、有良好团队协作能力；突出的开拓、共赢思维，高度的责任心。',
 '岗位职责:25-35岁，建筑、工民建、土木工程类相关专业，本科及以上学历，3年以上土建相关领域施工工作经验，最好有计划节点管控经验。',
 '岗位职责：1、对本专业图纸能独立进行会审；2、能过独立编制本专业的施工进度计划，并能独立协调现场发生的各类问题；3、熟悉开发项目的重点，工作中有预见性、大局观，与各专业工程师协调好；4、参与组织各项验收，督促检查工程质量、进度、安全；5、掌握天津市相关政策法规。任职要求：1、大专以上学历，土木工程相关专业，6年以上相关工作经验；2、熟悉和掌握土建专业的技能，熟练运用CAD和0ffice软件，能制作幻灯片，会运用projece排施工计划；3、有较强的组织能力、口头表达能力好、复合型人才优先、有较强的识图画图能力；特别注意：本职位为我公司所属塘沽项目，工作地点在森林公园附近。',
 '工作职责1、根据公司的经营目标，制定本部门的年度和阶段性工作计划。2、对本部门的日常工作进行管理、安排、监督、检查反馈各员工的工作的落实情况，处理反映的问题。3、协调相关单位处理施工中出现的技术和质量问题；4、组织本部门对工程的进度、质量、安全进行检查，严把工程各环节质量、进度、安全关；5、组织图纸审查工作，对设计方案和施工图进行审核、对各种材料的选用进行把关；6、负责组织工程项目初步验收及竣工验收工作把好验收质量关；7、组织协调相关工程的招标或委托工作；熟悉掌握每项工程动态，及时处理突发事件；任职资格1、大学本科及以上学历，建筑、土木、结构类相关专业；2、8年以上工程技术工作经验